# Palmer Penguins
This notebook is my attempt to analyze palmer's penguins dataset.

In [3]:
# Data Frames.
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
#Load the Penguins dataset.
df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv")


In [31]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [6]:
# Information about dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [14]:
# checking missing values
df.isnull().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

In [21]:
# removing columns with missing values
df.dropna(axis=0, inplace=True)
df.isnull().sum()

species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

In [37]:
cov_coeff=df.cov(numeric_only=True)
print(cov_coeff)

                   bill_length_mm  bill_depth_mm  flipper_length_mm  \
bill_length_mm          29.807054      -2.534234          50.375765   
bill_depth_mm           -2.534234       3.899808         -16.212950   
flipper_length_mm       50.375765     -16.212950         197.731792   
body_mass_g           2605.591912    -747.370093        9824.416062   

                     body_mass_g  
bill_length_mm       2605.591912  
bill_depth_mm        -747.370093  
flipper_length_mm    9824.416062  
body_mass_g        643131.077327  


In [38]:
corr_coeff=df.corr(numeric_only=True)
print(corr_coeff)

                   bill_length_mm  bill_depth_mm  flipper_length_mm  \
bill_length_mm           1.000000      -0.235053           0.656181   
bill_depth_mm           -0.235053       1.000000          -0.583851   
flipper_length_mm        0.656181      -0.583851           1.000000   
body_mass_g              0.595110      -0.471916           0.871202   

                   body_mass_g  
bill_length_mm        0.595110  
bill_depth_mm        -0.471916  
flipper_length_mm     0.871202  
body_mass_g           1.000000  


***
### End